In [28]:
!pip install pyqtgraph
!pip install PySide6
!pip install PyQt6
!pip install PyQt5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 31.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 4.5 MB/s eta 0:00:00


In [4]:
import numpy as np
import sys
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QPen, QBrush
from PyQt5.QtCore import Qt, QPoint
from IPython import get_ipython

In [5]:
def cubic_hermite(P0, P1, M0, M1, t):
    """
    Given a pair of points and a pair of vectors, compute the cubic Hermite
    polynomial they define and evaluate it at time t.
    """
    h00 = 2*t**3 - 3*t**2 + 1
    h10 = t**3 - 2*t**2 + t
    h01 = -2*t**3 + 3*t**2
    h11 = t**3 - t**2
    return h00 * P0 + h10 * M0 + h01 * P1 + h11 * M1
def estimate_derivatives_catmull_rom(P):
    """
    Assuming P are the points of a closed curve, use the Catmull-Rom
    technique to estimate the derivative at each point of P by looking at its
    neighbors.
    """
    derivatives = np.zeros_like(P, dtype=float)
    derivatives[1:-1] = (P[:-2] - P[2:]) / 2
    derivatives[-1] = (P[-2]-P[0])/2
    derivatives[0] = (P[-1] - P[1])/2
    return derivatives
def catmull_rom_interpolation(P, t):
    """
    P is a set of points, which we assume are corresponded to equally spaced
    times between 0 and 1. t is a scalar value between 0 and 1. Return the
    value of the catmull-rom interpolation of P at time t. You may use
    cubic_hermite.
    """
    derivatives=estimate_derivatives_catmull_rom(P)
    n = P.shape[0]-1  # Integer division to get the number of segments    nt=n*t;
    nt=n*t;
    nt,seg=np.modf(nt)
    seg=seg.astype(int)
    p0=P[seg-1]
    p1=P[seg]
    m0=derivatives[seg-1]
    m1=derivatives[seg]
    return cubic_hermite(p0,p1,m0,m1,nt);
    pass


In [38]:

"""randompoints = np.array([[25,25],
                        [25,100],
                        [45, 125]])
newlist = []

for i in th:
    #print(catmull_rom_interpolation(randompoints, (1-i)))
    newvertex = catmull_rom_interpolation(randompoints, (1-i))
    newlist.append(QPoint(newvertex[0].astype(int), newvertex[1].astype(int)))"""
    

def curvepositions(Qplist):
    points = []
    #points = np.empty((0,2), dtype=int)
    newlist = []
    th = np.linspace(0, 1, 1000)
    for i in Qplist:
        points.append([i.x(), i.y()])
    array = np.array((points))
    for i in th:
        newvertex = catmull_rom_interpolation(array, (1-i))
        newlist.append(QPoint(newvertex[0].astype(int), newvertex[1].astype(int)))
    return newlist


In [ ]:


class VertexWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.vertices = [QPoint(20, 20), QPoint(40, 100)]
        self.selected_vertex_index = None
        self.setWindowTitle("Vertex Mover")
        self.setGeometry(100, 100, 400, 300)
        self.show()
    def paintEvent(self, event):
        painter = QPainter(self)
        painter.setRenderHint(QPainter.Antialiasing)
        painter.setPen(QPen(Qt.black, 2))
        painter.setBrush(QBrush(Qt.blue, Qt.SolidPattern))
        for vertex in self.vertices:
            painter.drawEllipse(vertex, 10, 10)
        #for vertexnumber in range (len(self.vertices)-1):
            #painter.drawLine(self.vertices[vertexnumber], self.vertices[vertexnumber+1])
        newlist = curvepositions(self.vertices)
        for vertexnumber in range (len(newlist)-1):
            painter.drawLine(newlist[vertexnumber], newlist[vertexnumber+1])
    def mousePressEvent(self, event):
        for index, vertex in enumerate(self.vertices):
            if (event.pos() - vertex).manhattanLength() < 10:
                self.selected_vertex_index = index
                break
        if event.button() == Qt.MouseButton.RightButton:
            print('Hello')
            self.vertices.append(event.pos())
            self.update()
    def keyPressEvent(self, event):
        key = event.key()
        if key == Qt.Key_Space:
            print('space')
            self.vertices = self.vertices[:-1]
            self.update()
    def mouseMoveEvent(self, event):
        if self.selected_vertex_index is not None:
            self.vertices[self.selected_vertex_index] = event.pos()
            self.update()
    def mouseReleaseEvent(self, event):
        self.selected_vertex_index = None
def run_app():
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    widget = VertexWidget()
    app.exec_()




# Use the '%matplotlib qt' magic to enable the Qt event loop integration
get_ipython().run_line_magic('matplotlib', 'qt')
# Run the app
run_app()